In [542]:
%matplotlib inline

In [570]:
import hvplot.pandas

In [578]:
import pandas as pd
import numpy as np
from fastcore import *
from fastcore.all import * 

In [579]:
base_path = Path('/mnt/c/trial/results')

In [580]:
experiment_paths = base_path.ls()
experiment_paths

(#3) [Path('/mnt/c/trial/results/matan-lr-rsr-locality'),Path('/mnt/c/trial/results/new-minisat-localrestart'),Path('/mnt/c/trial/results/original')]

In [581]:
# import os
# for f_p in Path('/mnt/c/trial/results/matan-lr-rsr-locality').ls():
#     f_name = str(f_p)
#     os.rename(f_name, f_name.replace('Task-',''))

In [582]:
for p in experiment_paths:
    print (p, len(p.ls()))

/mnt/c/trial/results/matan-lr-rsr-locality 394
/mnt/c/trial/results/new-minisat-localrestart 398
/mnt/c/trial/results/original 397


In [585]:
new_names = [p.name for p in Path('/mnt/c/trial/results/new-minisat-localrestart').ls()]
original = [p.name for p in Path('/mnt/c/trial/results/original').ls()]
lr = [p.name for p in Path('/mnt/c/trial/results/matan-lr-rsr-locality').ls()]


rel_names = list(set(new_names).intersection(set(original)).intersection(set(lr)))

In [595]:
def read_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.read().splitlines()
        
    d = {}
    for l in lines:
        value = l.split(':')
        d[value[0]] = value[1]
    
    return d

In [602]:
def read_experiment_results(name):
    exp = base_path / name
    df = pd.DataFrame([read_file(file_name) for file_name in exp.ls() if file_name.name in names])
    df['experiment'] = name
    return df

In [651]:
a = experiments[0]
experiment_names = [p.name for p in experiment_paths]
experiment_names

['matan-lr-rsr-locality', 'new-minisat-localrestart', 'original']

In [652]:
df = pd.concat([read_experiment_results(name) for name in experiment_names], axis=0)

In [653]:
df['num_backtracks'] = df['num_backtracks'].fillna('0')
df['num_restarts'] = df['num_restarts'].fillna('0')
df['vdh'] = df['vdh'].fillna('MiniSAT')
df['solver'] = df['solver'].fillna('edusat')
df.loc[df['experiment']=='matan-lr-download','vdh']='LR'
df = df[df['experiment']!='matan-lr-download-old']
# df = df[df['filename'].str.replace('.txt','').isin([name.replace('.txt','') for name in rel_names])

In [654]:
df = df.astype({'nclauses': int, 'num_assignments': int, 'num_backtracks': int, 'num_decisions': int, 'num_learned': int, 
                    'nvars': int, 'runtime': float})

In [655]:
df.shape

(1186, 15)

In [656]:
df.head(3).T

,0,1,2
filename,3col200_5_3.shuffled.cnf,3col200_5_5.shuffled.cnf,3col20_5_10.shuffled.cnf
nclauses,1706,1706,176
num_assignments,15105700,17585937,265
num_backtracks,141391,99054,25
num_decisions,443767,1123054,70
num_learned,141391,99054,25
num_restarts,17485,65377,6
nvars,400,400,40
runtime,1024.669,1200.01,0.0
solver,new_solver,new_solver,new_solver


# Check for errors

In [657]:
df.groupby(['experiment', 'error']).agg({'experiment':'count'})

experiment
experiment               error                                           
matan-lr-rsr-locality    Found conflict at level dl=0                 116
                         Timeout of **1200** seconds exceeded          31
new-minisat-localrestart Found conflict at level dl=0                 125
                         Timeout of **1200** seconds exceeded           4

Great! No runtime errors! Only conflicts and timeouts!

# Statistics per experiment

In [658]:
df.groupby('experiment').count()

,filename,nclauses,num_assignments,num_backtracks,num_decisions,num_learned,num_restarts,nvars,runtime,solver,status,timeout,vdh,error
experiment,,,,,,,,,,,,,,
matan-lr-rsr-locality,392,392,392,392,392,392,392,392,392,392,392,392,392,147
new-minisat-localrestart,397,397,397,397,397,397,397,397,397,397,397,397,397,129
original,397,397,397,397,397,397,397,397,397,397,397,397,397,0


In [659]:
df.groupby(['experiment', 'solver', 'vdh', 'status']).agg({'experiment': 'count', 'runtime': 'mean',
                'num_assignments': 'mean', 'num_learned': 'mean', 'num_backtracks': 'mean', 'nclauses': 'mean'}).\
                    rename({'experiment': 'count'}, axis=1)

count      runtime  \
experiment               solver     vdh     status                        
matan-lr-rsr-locality    new_solver LR      ERROR       31  1200.011452   
                                            SAT        245    29.303388   
                                            UNSAT      116    39.472595   
new-minisat-localrestart new_solver MiniSAT ERROR        4  1200.007250   
                                            SAT        268    19.028612   
                                            UNSAT      125    34.156104   
original                 edusat     MiniSAT SAT        259    37.457591   
                                            TIMEOUT     19  1200.064737   
                                            UNSAT      119    26.359538   

                                                     num_assignments  \
experiment               solver     vdh     status                     
matan-lr-rsr-locality    new_solver LR      ERROR       2.385102e+07   
                                            SAT         1.156161e+06   
                                            UNSAT       1.738392e+06   
new-minisat-localrestart new_solver MiniSAT ERROR       2.346612e+07   
                                            SAT         2.150957e+06   
                                            UNSAT       3.132378e+06   
original                 edusat     MiniSAT SAT         8.866287e+10   
                                            TIMEOUT     2.131959e+12   
                                            UNSAT       3.708911e+10   

                                                       num_learned  \
experiment               solver     vdh     status                   
matan-lr-rsr-locality    new_solver LR      ERROR    149355.838710   
                                            SAT        5970.514286   
                                            UNSAT     13391.206897   
new-minisat-localrestart new_solver MiniSAT ERROR    114522.000000   
                                            SAT        5220.902985   
                                            UNSAT      8998.680000   
original                 edusat     MiniSAT SAT        3311.942085   
                                            TIMEOUT   62371.894737   
                                            UNSAT      5132.865546   

                                                     num_backtracks  \
experiment               solver     vdh     status                    
matan-lr-rsr-locality    new_solver LR      ERROR     149355.838710   
                                            SAT         5970.514286   
                                            UNSAT      13391.206897   
new-minisat-localrestart new_solver MiniSAT ERROR     114522.000000   
                                            SAT         5220.902985   
                                            UNSAT       8998.680000   
original                 edusat     MiniSAT SAT            0.000000   
                                            TIMEOUT        0.000000   
                                            UNSAT          0.000000   

                                                        nclauses  
experiment               solver     vdh     status                
matan-lr-rsr-locality    new_solver LR      ERROR    9042.870968  
                                            SAT      4403.118367  
                                            UNSAT    4187.543103  
new-minisat-localrestart new_solver MiniSAT ERROR    5145.250000  
                                            SAT      4796.608209  
                                            UNSAT    4351.208000  
original                 edusat     MiniSAT SAT      4732.131274  
                                            TIMEOUT  7493.736842  
                                            UNSAT    4050.168067

In [650]:
df[df['status']=='ERROR'][['experiment','filename','vdh','status','runtime']]

,experiment,filename,vdh,status,runtime


In [541]:
df[df['filename']=='unif-c1200-v300-s1582947682.cnf'][['experiment','filename','vdh','status','runtime']]

,experiment,filename,vdh,status,runtime
312,matan-lr-download,unif-c1200-v300-s1582947682.cnf,LR,ERROR,1200.001
312,new-minisat-localrestart,unif-c1200-v300-s1582947682.cnf,MiniSAT,SAT,46.938
312,original,unif-c1200-v300-s1582947682.cnf,MiniSAT,TIMEOUT,1200.040


In [566]:
df[df['experiment']=='new-minisat-localrestart'].sort_values('runtime', ascending=False).head(20)

,error,filename,nclauses,num_assignments,num_backtracks,num_decisions,num_learned,num_restarts,nvars,runtime,solver,status,timeout,vdh,experiment
112,Timeout of **1200** seconds exceeded,bart30.shuffled.cnf,3617,5394036,114745,253828,114745,9614,485,1200.011,new_solver,ERROR,1200,MiniSAT,new-minisat-localrestart
111,Timeout of **1200** seconds exceeded,bart29.shuffled.cnf,3248,9428624,43416,562931,43416,34143,456,1200.008,new_solver,ERROR,1200,MiniSAT,new-minisat-localrestart
171,Timeout of **1200** seconds exceeded,ezfact48_10.shuffled.cnf,11001,74845548,149442,353258,149442,11348,1729,1200.005,new_solver,ERROR,1200,MiniSAT,new-minisat-localrestart
108,Timeout of **1200** seconds exceeded,bart26.shuffled.cnf,2715,4196283,150485,189887,150485,2099,405,1200.005,new_solver,ERROR,1200,MiniSAT,new-minisat-localrestart
173,Found conflict at level dl=0,ezfact48_4.shuffled.cnf,11001,67566787,181880,397323,181880,10608,1729,1158.552,new_solver,UNSAT,1200,MiniSAT,new-minisat-localrestart
270,Found conflict at level dl=0,pyhala-braun-unsat-30-4-01.shuffled.cnf,17845,140409555,51454,288185,51454,39986,5428,841.556,new_solver,UNSAT,1200,MiniSAT,new-minisat-localrestart
146,NaN,dp12s12.shuffled.cnf,33520,166698149,75301,583193,75301,31830,12065,809.699,new_solver,SAT,1200,MiniSAT,new-minisat-localrestart
110,NaN,bart28.shuffled.cnf,2907,7629525,32670,482341,32670,29950,428,597.086,new_solver,SAT,1200,MiniSAT,new-minisat-localrestart
269,NaN,pyhala-braun-sat-30-4-04.shuffled.cnf,17845,38746384,56354,100802,56354,3000,5428,554.271,new_solver,SAT,1200,MiniSAT,new-minisat-localrestart
267,NaN,pyhala-braun-sat-30-4-01.shuffled.cnf,17845,33815788,46234,83426,46234,2018,5428,543.817,new_solver,SAT,1200,MiniSAT,new-minisat-localrestart
